Created by Joan-Marc Fisa

- Numerai: [FisaGol](https://numer.ai/fisagol)

- Twitter: [@fisagol](https://twitter.com/fisagol)


In [43]:
!pip install numerapi
!pip install catboost
!pip install vecstack;

In [44]:
##################################################################
##################### LIBRARIES ##################################
##################################################################


In [45]:
import os
import gc
import csv
import sys
import glob
import time
from pathlib import Path
from multiprocessing import Pool

import numerapi

import scipy
import numpy as np
import pandas as pd
import tensorflow as tf
import random
import sklearn
from sklearn import (
    feature_extraction, feature_selection, decomposition, linear_model,
    model_selection, metrics, svm, preprocessing, utils
)
from sklearn.preprocessing import StandardScaler,MinMaxScaler, OrdinalEncoder, LabelEncoder,OneHotEncoder
from keras.models import Sequential, model_from_json, load_model
from keras.layers import Dense, Dropout, Activation,LSTM,Bidirectional, MaxPooling2D, Flatten,GRU
from keras.optimizers import SGD,Adam
from keras.regularizers import l2
from sklearn.model_selection import StratifiedKFold, GroupKFold, GridSearchCV,cross_val_score,KFold, RepeatedStratifiedKFold,train_test_split
from sklearn.metrics import log_loss, make_scorer, mean_squared_error,classification_report,accuracy_score
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils 
from sklearn import preprocessing
from xgboost import XGBRegressor 
from sklearn.cluster import KMeans
import matplotlib as plt
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler,MinMaxScaler

import math
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor, BaggingRegressor
from sklearn.linear_model import SGDRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.tree import DecisionTreeRegressor 

import torch.nn as nn
import torch.nn.functional as F
import torch
from vecstack import stacking

from sklearn import metrics

def RMSLE(y, pred):
    return metrics.mean_squared_error(y, pred) ** 0.5


In [49]:
##############################################################################
########################## DOWLOAD DATA ######################################
##############################################################################

In [50]:
TOURNAMENT_NAME = "nomi"
TARGET_NAME = f"target"
PREDICTION_NAME = f"prediction"

BENCHMARK = 0
BAND = 0.2

In [51]:
seed_1 = 1
rand_1 = np.random.seed(seed_1)
seed_2 = rand_1
rand_2 = np.random.seed(seed_2)
seed_3 = rand_2
rand_3 = np.random.seed(seed_3)

In [52]:
napi = numerapi.NumerAPI(verbosity="info")

napi.download_current_dataset(unzip=True)

current_ds = napi.get_current_round()
latest_round = os.path.join('numerai_dataset_'+str(current_ds))

2021-02-07 16:22:38,736 INFO numerapi.base_api: target file already exists


In [53]:
##################################################################
##################### LOAD DATA ##################################
##################################################################

In [54]:
%%time
print("# Loading data...")

training_data = pd.read_csv(os.path.join(latest_round, "numerai_training_data.csv")).set_index("id")
tournament_data = pd.read_csv(os.path.join(latest_round, "numerai_tournament_data.csv")).set_index("id")
validation_data = tournament_data[tournament_data.data_type == "validation"]

print("# All Loaded...")

# Loading data...
# All Loaded...
CPU times: user 1min 7s, sys: 17 s, total: 1min 24s
Wall time: 1min 25s


In [55]:
feature_names = [f for f in training_data.columns if f.startswith("feature")]
print(f"Loaded {len(feature_names)} features")

Loaded 310 features


In [56]:
X = training_data[feature_names]
Y = training_data[training_data.columns[-1]]

le = LabelEncoder()
Y_enc = le.fit_transform(Y)

In [60]:
#############################################################################
###########################  CREATING SOME MODELS  ##########################
#############################################################################

In [61]:
%%time

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=rand_2)

model_x = [
          
    LGBMRegressor(random_state=rand_1),
    LGBMRegressor(random_state=rand_2),
    LGBMRegressor(random_state=rand_3)
]

from sklearn.metrics import explained_variance_score

S_train, S_test = stacking(model_x,                   
                          X_train, y_train, X_test,   
                          regression=True, 
                          mode='oof_pred_bag',
                          needs_proba=False,
                          save_dir=None,
                          metric= explained_variance_score,
                          n_folds=7,
                          stratified=True,
                          shuffle=True,
                          random_state=rand_3,verbose=2)


del model_x

gc.collect()

/usr/local/lib/python3.6/dist-packages/vecstack/core.py:461: UserWarning: This is regression task hence classification-specific parameters set to <True> were ignored: <stratified>
  warnings.warn(warn_str, UserWarning)


task:         [regression]
metric:       [explained_variance_score]
mode:         [oof_pred_bag]
n_models:     [3]

model  0:     [LGBMRegressor]
    fold  0:  [0.00159755]
    fold  1:  [0.00154858]
    fold  2:  [0.00132814]
    fold  3:  [0.00098543]
    fold  4:  [0.00185538]
    fold  5:  [0.00110849]
    fold  6:  [0.00142460]
    ----
    MEAN:     [0.00140688] + [0.00027536]
    FULL:     [0.00140588]

model  1:     [LGBMRegressor]
    fold  0:  [0.00261561]
    fold  1:  [0.00116105]
    fold  2:  [0.00102212]
    fold  3:  [0.00133278]
    fold  4:  [0.00126586]
    fold  5:  [0.00235270]
    fold  6:  [0.00151227]
    ----
    MEAN:     [0.00160891] + [0.00057506]
    FULL:     [0.00160454]

model  2:     [LGBMRegressor]
    fold  0:  [0.00168942]
    fold  1:  [0.00080197]
    fold  2:  [0.00152772]
    fold  3:  [0.00204758]
    fold  4:  [0.00130116]
    fold  5:  [0.00129662]
    fold  6:  [0.00177624]
    ----
    MEAN:     [0.00149153] + [0.00037425]
    FULL:     [0.0

In [62]:
model_e = LGBMRegressor(random_state=rand_1)
gc.collect()

0

In [63]:
model_1 = LGBMRegressor(random_state=rand_1)
model_1.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

36

In [64]:
model_2 = LGBMRegressor(random_state=rand_2)
model_2.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

36

In [65]:
model_3 = LGBMRegressor(random_state=rand_3)
model_3.fit(training_data[feature_names], training_data[TARGET_NAME])
gc.collect()

36

In [66]:
y_pred_model_1 = model_1.predict(tournament_data[feature_names])
gc.collect()

4

In [67]:
y_pred_model_2 = model_2.predict(tournament_data[feature_names])
gc.collect()

4

In [68]:
y_pred_model_3 = model_3.predict(tournament_data[feature_names])
gc.collect()

4

In [69]:
df_mix = pd.DataFrame(columns=['f0','f1','f2'])
df_mix['f0'] = y_pred_model_1
df_mix['f1'] = y_pred_model_2
df_mix['f2'] = y_pred_model_3

In [70]:
df_mix

,f0,f1,f2
0,0.488480,0.488480,0.488480
1,0.487922,0.487922,0.487922
2,0.516811,0.516811,0.516811
3,0.499933,0.499933,0.499933
4,0.502267,0.502267,0.502267
...,...,...,...
1644410,0.489726,0.489726,0.489726
1644411,0.494198,0.494198,0.494198
1644412,0.494413,0.494413,0.494413
1644413,0.510319,0.510319,0.510319


In [71]:
model_e = model_e.fit(S_train, y_train)
y_pred = model_e.predict(S_test)
gc.collect()

45

In [72]:
y_pred_final = model_e.predict(df_mix)
tournament_data[PREDICTION_NAME] = y_pred_final
gc.collect()

4

In [73]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.485133
n000920ed083903f    0.481120
n0038e640522c4a6    0.523189
n004ac94a87dc54b    0.499174
n0052fe97ea0c05f    0.501162
                      ...   
nffe93b9f7d7530d    0.486422
nffef5e57638d46e    0.491013
nfff640225894e11    0.491488
nfff6e21335c357f    0.516860
nfffc6ad6d850c65    0.473996
Name: prediction, Length: 1644415, dtype: float64

In [74]:
import numerapi
import torch
from torch.nn import Linear
from torch.nn import Sequential
from torch.functional import F

In [75]:
def exposures(x, y):
    x = x - x.mean(dim=0)
    x = x / x.norm(dim=0)
    y = y - y.mean(dim=0)
    y = y / y.norm(dim=0)
    return torch.matmul(x.T, y)

def reduce_exposure(prediction, features, max_exp):
    # linear model of features that will be used to partially neutralize predictions
    lin = Linear(features.shape[1],  1, bias=False)
    lin.weight.data.fill_(0.)
    model = Sequential(lin)
    optimizer = torch.optim.Adamax(model.parameters(), lr=1e-4)
    feats = torch.tensor(np.float32(features)-.5)
    pred = torch.tensor(np.float32(prediction))
    start_exp = exposures(feats, pred[:,None])
    # set target exposure for each feature to be <= current exposure
    # if current exposure is less than max_exp, or <= max_exp if  
    # current exposure is > max_exp
    targ_exp = torch.clamp(start_exp, -max_exp, max_exp)

    for i in range(100000):
        optimizer.zero_grad()
        # calculate feature exposures of current linear neutralization
        exps = exposures(feats, pred[:,None]-model(feats))
        # loss is positive when any exposures exceed their target
        loss = (F.relu(F.relu(exps)-F.relu(targ_exp)) + F.relu(F.relu(-exps)-F.relu(-targ_exp))).sum()
        print(f'       loss: {loss:0.7f}', end='\r')
        if loss < 1e-7:
            neutralizer = [p.detach().numpy() for p in model.parameters()]
            neutralized_pred = pred[:,None]-model(feats)
            break
        loss.backward()
        optimizer.step()
    return neutralized_pred, neutralizer

def reduce_all_exposures(df, column, neutralizers=[],
                                     normalize=True,
                                     gaussianize=True,
                                     era_col="era",
                                     max_exp=0.1):
    unique_eras = df[era_col].unique()
    computed = []
    for u in unique_eras:
        print(u, '\r')
        df_era = df[df[era_col] == u]
        scores = df_era[column].values
        exposure_values = df_era[neutralizers].values
        
        if normalize:
            scores2 = []
            for x in scores.T:
                x = (scipy.stats.rankdata(x, method='ordinal') - .5) / len(x)
                if gaussianize:
                    x = scipy.stats.norm.ppf(x)
                scores2.append(x)
            scores = np.array(scores2)[0]

        scores, neut = reduce_exposure(scores, exposure_values, max_exp)

        scores /= scores.std()

        computed.append(scores.detach().numpy())

    return pd.DataFrame(np.concatenate(computed), columns=column, index=df.index)

In [76]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.485133
n000920ed083903f    0.481120
n0038e640522c4a6    0.523189
n004ac94a87dc54b    0.499174
n0052fe97ea0c05f    0.501162
                      ...   
nffe93b9f7d7530d    0.486422
nffef5e57638d46e    0.491013
nfff640225894e11    0.491488
nfff6e21335c357f    0.516860
nfffc6ad6d850c65    0.473996
Name: prediction, Length: 1644415, dtype: float64

In [77]:
data_rfe_10 = reduce_all_exposures(tournament_data,
                                   [PREDICTION_NAME],
                                   neutralizers=feature_names,
                                   era_col="era",
                                   max_exp=0.10)

era121 
era122 
era123 
era124 
era125 
era126 
era127 
era128 
era129 
era130 
era131 
era132 
era575 
era576 
era577 
era578 
era579 
era580 
era581 
era582 
era583 
era584 
era585 
era586 
era587 
era588 
era589 
era590 
era591 
era592 
era593 
era594 
era595 
era596 
era597 
era598 
era599 
era600 
era601 
era602 
era603 
era604 
era605 
era606 
era607 
era608 
era609 
era610 
era611 
era612 
era613 
era614 
era615 
era616 
era617 
era618 
era619 
era620 
era621 
era622 
era623 
era624 
era625 
era626 
era627 
era628 
era629 
era630 
era631 
era632 
era633 
era634 
era635 
era636 
era637 
era638 
era639 
era640 
era641 
era642 
era643 
era644 
era645 
era646 
era647 
era648 
era649 
era650 
era651 
era652 
era653 
era654 
era655 
era656 
era657 
era658 
era659 
era660 
era661 
era662 
era663 
era664 
era665 
era666 
era667 
era668 
era669 
era670 
era671 
era672 
era673 
era674 
era675 
era676 
era677 
era678 
era679 
era680 
era681 
era682 
era683 
era684 
era685 
era686 
era687 


In [78]:
# replace prediction with reduced feature exposure prediction and rescale to [0,1]
tournament_data[PREDICTION_NAME] = data_rfe_10[PREDICTION_NAME]
tournament_data[PREDICTION_NAME] -= tournament_data[PREDICTION_NAME].min()
tournament_data[PREDICTION_NAME] /= tournament_data[PREDICTION_NAME].max()

In [79]:
tournament_data[PREDICTION_NAME]

id
n0003aa52cab36c2    0.422881
n000920ed083903f    0.383406
n0038e640522c4a6    0.551145
n004ac94a87dc54b    0.537376
n0052fe97ea0c05f    0.557586
                      ...   
nffe93b9f7d7530d    0.512587
nffef5e57638d46e    0.448285
nfff640225894e11    0.543187
nfff6e21335c357f    0.667225
nfffc6ad6d850c65    0.277757
Name: prediction, Length: 1644415, dtype: float64

In [80]:
##############################################################################
######################### MAKE SUBMISSION #################################### 
##############################################################################

In [81]:
tournament_data[PREDICTION_NAME].to_csv(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv")

In [82]:
# NameOfYourAI FISAGOL_1: joanmarc92@hotmail.com
public_id = "3LNFQEKJM2EFGHLRC5WK344GZTE3Y2XK"
secret_key = "CCMP4MU2YDUZQYNO5BUZY6SQHX3NKBPSZDECBBFM6TLNCLY4VISKQ6Z6CBZGJ65Y"
model_id = "0bb6b078-8525-4521-9ed2-5e7d13cc88ca"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

'\n# NameOfYourAI FISAGOL_1: joanmarc92@hotmail.com\npublic_id = "3LNFQEKJM2EFGHLRC5WK344GZTE3Y2XK"\nsecret_key = "CCMP4MU2YDUZQYNO5BUZY6SQHX3NKBPSZDECBBFM6TLNCLY4VISKQ6Z6CBZGJ65Y"\nmodel_id = "0bb6b078-8525-4521-9ed2-5e7d13cc88ca"\nnapi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)\n'

In [83]:
print("Uploading DataFrame in Numerai...")
submission_id = napi.upload_predictions(f"{TOURNAMENT_NAME}_{current_ds}_submission.csv", model_id=model_id)
print("DataFrame Uploaded...")

2021-02-07 19:45:59,650 INFO numerapi.base_api: uploading predictions...


Uploading DataFrame in Numerai...
DataFrame Uploaded...


In [94]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


In [100]:
tournament_data[PREDICTION_NAME].to_csv('FisaGol_1.csv')
!cp FisaGol_1.csv "drive/My Drive/"

In [84]:
'''
##########################################################################################
##############################   SAVE MODEL   ############################################
##########################################################################################
from sklearn.externals import joblib
joblib.dump(my_model, "my_model.pkl")
# and later...
my_model_loaded = joblib.load("my_model.pkl")
'''

'\n##########################################################################################\n##############################   SAVE MODEL   ############################################\n##########################################################################################\nfrom sklearn.externals import joblib\njoblib.dump(my_model, "my_model.pkl")\n# and later...\nmy_model_loaded = joblib.load("my_model.pkl")\n'